<a href="https://colab.research.google.com/github/scskalicky/LING-226-vuw/blob/main/08_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The importance of pre-processing



It's time to return to something we've already covered — tokenizing a text and defining what counts as a word. So far we've already been doing this with the `.split()` function, which has worked relatively well for us. But, there is one issue, which is that splitting on white space means that sometimes punctuation is included with our words.

For example, running `.split()` on the example below will retain commas and exclamation marks as part of the words:






In [ ]:
turtles = """teenage mutant ninja turtles,
            teenage mutant ninja turtles,
            teenage mutant ninja turtles,
            heroes in a halfshell, turtle power!"""

turtles.split()

Therefore, we might want to perform some operations on this text *before* we start processing it for linguistic information. These operations will work to normalize and standardize the text so that noise is removed. This is called preprocessing. Preprocessing comes in many options - you could remove just punctuation, or convert everything to lowercase, or remove very frequent words, or remove words that are not in the dictionary, or remove words that only occur one time, and so on. Different algorithms and approaches to NLP will all include their own methods and steps for preprocessing, which are tied to the goals of the analysis.

For now, let's focus on the issue of punctuation in the turtles text.

### Frequency and pre-processing

What will happen if we run `.split()` and create a `FreqDist` from the turtles text without any preprocessing?


Let's import the NLTK resources first...


In [ ]:
# import the main nltk module
import nltk

# download the nltk.book resources
nltk.download('book')

# import the resources
from nltk.book import *

In [ ]:
# make a frequency distro of our turtles
tfdist = nltk.FreqDist(turtles.split())

In [ ]:
# we know that the world "turtles" occurs in the song, so why don't we see it?
tfdist['turtles']

In [ ]:
# because the commas has been saved as part of the word! uhg!
tfdist['turtles,']

Using `.split()` clearly needs some help and introduces a fundamental topic in NLP and corpus linguistics — preprocessing or normalising a text.

Why is this important? Well, if we want to calculate the frequency of a word in a corpus / text *properly*, we have to make sure all words are on an even playing ground. Before we even get into punctuation, consider the following:

In [ ]:
nltk.FreqDist('Victoria University of WELLINGTON is in Wellington'.split())

Although the word "Wellington" occured twice in the string above, one version was in all capitals and one was not. The `FreqDist` function treated these as two separate words. Why? The answer reminds us about the way these strings are being compared by Python:

In [ ]:
# These are two different values!
'WELLINGTON' == 'Wellington'

While we know that these are basically the same word, Python doesn't care because they are *not* the same word in terms of being 100% identical values. So, we want to consider performing some initial processing (i.e., *preprocessing*) on a text before counting the words as a means to normalize or control for these properties of words we might not care about. For example, we could solve the problem above by converting all of our words to lower case.

In [ ]:
# Hey we're the same now!
'WELLINGTON'.lower() == 'Wellington'.lower()

### Lexical diversity and pre-processing.

As another example, let's consider how pre-processing influences the effects of a measure we've already explored: lexical diversity. Compare what capitalization will do to measures of lexical diversity on these two texts:

In [ ]:
# create two texts that only differ based on capitalization
version1 = ['Soda', 'soda', 'Onion', 'onion']
version2 = ['soda', 'soda', 'onion', 'onion']

In [ ]:
# remember how to measure ttr?
def lexical_diversity(text):
  ld = len(set(text))/len(text)
  return ld

Preprocessing leads to very different TTRs values for the "same" texts.

In [ ]:
lexical_diversity(version1)

In [ ]:
lexical_diversity(version2)

We clearly would not want to think that `version1` is more lexically diverse than `version2`, unless we have strong reason to believe the capitalization results in a fundamentally different word.

Hence, normalization is needed to address these issues.

You might question this approach and wonder whether normalizing serves to remove important information about a text - perhaps capitalization matters? What if Soda is a proper name and soda is just the noun?

These are important things to take into consideration when doing any sort of NLP - the scope of your research questions and the nature of the linguistic features you are interested in (and how you measure them) should drive these decisions.




### Cleaning punctuation

But our problem above with `turtles` was also caused by the use of punctuation and `.split()`. What could we do? Well, we *could* remove all of the punctuation before splitting the text, and this would provide a satisfactory solution (for now).

Based on what we know now about Python, how could we remove all of the punctuation from a text? We can actually do this quite simply and quickly using a list comprehension.

We would want to set a condition that inspects each character in a string, and as long as that character is *not* a punctuation mark, keep it.

Here is some pseudocode that expresses our goal:


```
[character for character in string if character not punctuation]
```

To exectute this code, we'd need to tell Python what we mean by "punctuation". One way is to define a string containing all the puncuation marks we don't want.

At the same time, we can make sure to lower case everything in the same expression.


In [ ]:
# define a string containing punctuation we don't like, in this case just commas and exclamation marks
punctuation = ',!'

In [ ]:
# write a list comprehension that only keeps characters that aren't in punctuation
# read on to the next section to see how to fix this output!
[character.lower() for character in turtles if character not in punctuation]

### `.join()`

Hrmm, not quite what we wanted, because the list comprehension has returned a list of *characters*, but we wanted to retain the whitespace and other properties of the texts. No worries, we can use the handy `.join()` function to join a list of characters back into one string!

`.join` is sort of the bizzare cousin of `.split()`. `.join` is actually a string method, meaning you need to attach a string to the front part of the `.join()`. The string that you attach to `.join` represents the nature of the join...the character that you want to join everything by. Much like `.split()`, you can choose whatever you like to join stuff with.

But, if we simply wanted to glue back together a list of characters *without* making any other changes, we would then attach an empty string to `.join()`, indicated with two string delimiters: `''`, in which case we would type `''.join()`.

Then, the thing that you want to join goes inside the `()` part of `''.join()`.

```
''.join([list of characters])
```


In [ ]:
# we just wrap the whole list comprehension in ''.join
remove_punctuation = ''.join([character.lower() for character in turtles if character not in punctuation])

In [ ]:
# it looks different now...but it's been reformed back into what we first had without punctuation
remove_punctuation

How else could we do this without using join?

One way would be to write a loop which analyses each word in a text, removing punctuation from that word, and then puts that word into a list. This is made slightly difficult because strings are `immutable`, meaning that we cannot remove or replace individual elements of a string.


In [ ]:
# this returns an error because we cannot modify strings in place
'string'[0] = 'b'

One way to do this is scan through each character and then reconstruct the string as we go, only including characters that pass the test.

String concatenation can be used for this, which is just a fancy way of saying that you can add two strings together to make a larger string.



In [ ]:
# create an output container
output = ''

# loop through each character in the whole string
for character in turtles:
  # if the character is NOT in this list:
  if character not in [',', '!']:
    # add the lowercased version of the character to the list
    output = output + character.lower()

# results are identical to the ''.join() method above
output


Regardless of the method used to preprocess the text, the resulting `FreqDist` will now look a bit different.

In [ ]:
# create a new frequency distribution
cleaned_fdist = nltk.FreqDist(remove_punctuation.split())

In [ ]:
# all the punctuation is gone, and all words are lowercased
cleaned_fdist

In [ ]:
# now we get proper results for turtles
cleaned_fdist['turtles']

## `nltk.word_tokenize()`

Now we have a better way to use `.split()` and understand some potential solutions for the way that punctuation may interfere with our definition of words.

However - what if we wanted to retain punctuation? Do you think it would be important to know the difference between words that come before / after punctuation? Could punctuation tell us something about the syntax of a sentence or the tone of voice of writing? These are questions without clear answers, but are worthy of consideration. Another more practical aspect of retaining punctuation is that punctuation markers could help with segmentation of strings into words and/or sentences. For this reason, using `.split()` can use some help.

NLTK has two built-in segmentation functions which are improvements upon using `.split()`. These function are `nltk.word_tokenize()` and `nltk.sent_tokenize()`. They convert raw strings into tokens or sentences, respectively. Let's just focus on word tokenization for now.

In the cells below, compare the difference between using `.split()` and `nltk.word_tokenize()`:

In [ ]:
# What is the difference between using `.split()` and `nltk.word_tokenize()`?
pretzels = 'These pretzels are making me thirsty!'

split_tokens = pretzels.split()
nltk_tokens = nltk.word_tokenize(pretzels)

print(f"Using .split(): \n{split_tokens}\n\nUsing nltk: \n{nltk_tokens}")

The NLTK tokenizer has treated the punctuation as a separate word - so it is smart enough to recognise that words should be separated from punctuation. It does this using a set of additional rules as well as some splitting. This makes perfect sense for punctuation which occurs after words, such as commas, full stops, exclamation marks, and so on.

What's going on in the cell below?

In [ ]:
# What is different about these tokens?
nltk.word_tokenize('I can\'t even.')

The word "can't" was split into two tokens! Why is that? Well, if we think about it, "can't" actually stands for *two* words - "can" and "not." The tokenizer has an additional set of rules to search these contractions and split them accordingly. Using `.split()`, on the other hand, would result in "can't" being stored as a single word. Moreover, removing the punctuation *before* tokenization would turn "can't" into "cant", and then `nltk.word_tokenize()` would treat "cant" as a single word. Is this an issue? Well, considering the word "cant" is its own word separate in meaning from "can't", it certainly could be.


The point is that the order of pre-processing and normalisation steps is important, as are the different things you might want to do to a text. Many modern NLP libraries perform pre-processing automatically, and it is fundamental to understand how your data is being normalised in order to use these functions properly.

## **Your Turn**

Spend some time becoming familiar with the differences between `.split()` and `nltk.word_tokenize()`.

As part of your comparisons, create frequency distributions based on the results of `.split()` and `nltk.word_tokenize()` for the same strings.
